In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"],1)

/Users/Biggaharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Determine which values to replace if counts are less than 500
replace_application = list(app_counts[app_counts<500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than 1500
replace_class = list(class_counts[class_counts<1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
app_cat = application_df.dtypes[application_df.dtypes=="object"].index.tolist()

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

#Create the DataFrame, then fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(app_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, right_index = True, left_index = True)
application_df = application_df.drop(app_cat, 1)
application_df.head()

/Users/Biggaharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
#Remove target and binned features in effort to reduce noise
X = application_df.drop(columns=["IS_SUCCESSFUL","APPLICATION_TYPE_Other","CLASSIFICATION_Other"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### Attempt 1

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layers1 = 6
hidden_nodes_layers2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers1,input_dim=number_input_features,activation="relu")
)

#Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers2,activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 252       
                                                                 
 dense_1 (Dense)             (None, 5)                 35        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 293
Trainable params: 293
Non-trainable params: 0
_________________________________________________________________


2022-03-13 17:27:38.151898: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6167 - accuracy: 0.6577
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5806 - accuracy: 0.6979
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5713 - accuracy: 0.7252
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5671 - accuracy: 0.7258
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5645 - accuracy: 0.7259
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5629 - accuracy: 0.7267
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5618 - accuracy: 0.7260
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5608 - accuracy: 0.7252
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5598 - accuracy: 0.7263
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5594 - accuracy: 0.7260

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5557 - accuracy: 0.7265 - 348ms/epoch - 1ms/step
Loss: 0.5556774735450745, Accuracy: 0.7265306115150452


#### Attempt 2


In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layers1 = 12
hidden_nodes_layers2 = 6

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers1,input_dim=number_input_features,activation="tanh")
)

#Second hidden layer
nn_1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers2,activation="tanh")
)

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12)                504       
                                                                 
 dense_4 (Dense)             (None, 6)                 78        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 589
Trainable params: 589
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7334
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7327
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7336
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5501 - accuracy: 0.7328
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7333
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7329
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7325
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7323
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7330
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7334

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5561 - accuracy: 0.7252 - 325ms/epoch - 1ms/step
Loss: 0.5561450719833374, Accuracy: 0.7252478003501892


#### Attempt 3

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layers1 = 12
hidden_nodes_layers2 = 6
hidden_nodes_layers3 = 3

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers1,input_dim=number_input_features,activation="sigmoid")
)

# #Second hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers2,activation="sigmoid")
)

# #Third hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers3,activation="sigmoid")
)

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 12)                504       
                                                                 
 dense_7 (Dense)             (None, 6)                 78        
                                                                 
 dense_8 (Dense)             (None, 3)                 21        
                                                                 
 dense_9 (Dense)             (None, 1)                 4         
                                                                 
Total params: 607
Trainable params: 607
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os
# Define the checkpoint path and filenames
os.makedirs("checkpoints_2",exist_ok=True)
checkpoint_path = "checkpoints_2/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 4020)

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks =[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7334
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7338
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7339
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5489 - accuracy: 0.7341
Epoch 5/50
782/804 [============================>.] - ETA: 0s - loss: 0.5487 - accuracy: 0.7338
Epoch 5: saving model to checkpoints_2/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7341
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7336
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7350
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7333
Epoch 9/50
804/804 [==============================] - 1s 1ms

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5562 - accuracy: 0.7250 - 327ms/epoch - 1ms/step
Loss: 0.5562413930892944, Accuracy: 0.7250145673751831


In [25]:
#Export our model to HDF5 file 
nn.save("AlphabetSoupCharity_Optimization.h5")

### Summary testing 

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=256, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Random forest predictive accuracy: 0.709


In [34]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=10000)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.467
